In [1]:
import sys
sys.path.append("..")
sys.path.append("../src")

import yaml
config = yaml.load(open('../src/configs/config.yaml', 'r'))

/home/francisco/anaconda3/envs/covid/lib/python3.6/site-packages/ipykernel_launcher.py:6: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [2]:
%reload_ext autoreload
%autoreload 2


from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH, THEMES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
import geopandas as gpd
pd.options.display.max_columns = 999

import warnings

warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf

plotly.offline.init_notebook_mode(connected=True)

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Centering and fixing title
def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Adding custom colorscales (to add one: themes/custom_colorscales.yaml)
import yaml
custom_colorscales = yaml.load(open(THEMES_PATH / 'custom_colorscales.yaml', 'r'))
cf.colors._custom_scales['qual'].update(custom_colorscales)
cf.colors.reset_scales()

# Setting cuffilinks template (use it with .iplot(theme='custom')
cf.themes.THEMES['custom'] = yaml.load(open(THEMES_PATH / 'cufflinks_template.yaml', 'r'))

## Import data

In [3]:
df = pd.read_csv('http://localhost:80/br/cities/cases/full')

In [4]:
UF="CE"
t = df.query(f'state =="{UF}"')

## Process data

In this section, we selected some COVID features like state notification rate, last available death rate, deaths, but you may choose the data you may find necessary to analyse

In [5]:
tsn = t[~t['state_notification_rate'].isnull()][['last_updated','state','city','city_id','state_notification_rate', 'notification_rate']]
tsn = tsn.sort_values('city').reset_index(drop=True)
citycode = tsn[['city', 'city_id']].drop_duplicates().reset_index(drop=True)

In [6]:
def var_through_time(t,place_type,col_time,var,norm=False):
    if norm:
        df_tt = t.reset_index()\
                              .pivot(index=place_type, 
                                      columns=col_time, 
                                      values=var)\
                               .fillna(0)\
                               .apply(lambda x: x/x.max(), axis=1)\
                               .dropna(how='all')
    else:
        df_tt = t.reset_index()\
                              .pivot(index=place_type, 
                                      columns=col_time, 
                                      values=var)\
                               .fillna(0)\
                               .dropna(how='all')
    # remove days with all states zero
    df_tt = df_tt.loc[:, (df_tt != 0).any(axis=0)]
    return df_tt

In [7]:
#Get notification rate through time
vtt = var_through_time(tsn,'city','last_updated','notification_rate')

In [8]:
import geobr
import matplotlib.pyplot as plt
%matplotlib inline
import json
import geojsonio
from plotly.offline import download_plotlyjs, plot, iplot

## Loading geodata

In [9]:
year = 2018
types = {
    'Cities': geobr.read_municipality(code_muni=UF, year=year)
}

Sorting the data by states alphabetical order

In [10]:
df1 = types['Cities']
#Sort df1 by city name
df1 = df1.sort_values('name_muni').reset_index(drop=True)

In [11]:
#Convert city code to int
df1['code_muni'] = df1['code_muni'].astype(int)
df1.head()

,code_muni,name_muni,code_state,abbrev_state,geometry
0,2300101,Abaiara,23,CE,"POLYGON ((-39.10117 -7.34332, -39.10202 -7.328..."
1,2300150,Acarape,23,CE,"POLYGON ((-38.71787 -4.21601, -38.72994 -4.169..."
2,2300200,Acaraú,23,CE,"POLYGON ((-40.15508 -2.88291, -40.20002 -2.875..."
3,2300309,Acopiara,23,CE,"POLYGON ((-39.08967 -6.09431, -39.08456 -6.100..."
4,2300408,Aiuaba,23,CE,"POLYGON ((-40.72293 -6.62135, -40.72200 -6.617..."


In [12]:
gdf = gpd.GeoDataFrame(df1[['name_muni','geometry']])
jdf = json.loads(df1[['name_muni','geometry']].to_json())

In [13]:
#Get most recent notification rate
snr = tsn.query(f'last_updated=="{vtt.columns[-1]}"')['state_notification_rate'].iloc[0]

In [14]:
#Create a array with df1 shape and state notefication rate values
snrnp = np.ones(df1.shape[0])*snr

In [15]:
#Verify if citycode['city'] is in the same order of vtt
assert list(citycode['city'].values) == list(vtt.index)

idx = [pd.Index(df1['code_muni']).get_loc(i) for i in citycode['city_id'].values]

#If city have notification rate use city notification rate, else, use state notification rate 
snrnp[idx] = vtt[vtt.columns[-1]].values

In [16]:
cbar_title = 'notification rate (%)'
Title = f'Notification Rate {UF} in {vtt.columns[-1]}'
cmap = "temps_r"

In [17]:
plot_data = dict(type = 'choropleth',
                geojson=jdf,
                locations =gdf.index.astype(str),
                locationmode = 'geojson-id',
                colorscale= cmap,
                text= gdf['name_muni'].astype(str),
                z=snrnp.astype(float),
                zmin=0,zmax=1,
                colorbar = {'title':cbar_title})
    
layout = dict(title_text = Title,
              geo = {'scope':'south america',
                    'fitbounds':"locations",
                     'visible':False})
fig = dict(data = plot_data,layout = layout)

plotly.offline.iplot(fig)